<a href="https://colab.research.google.com/github/mammadhajili/nlp_tutorials/blob/main/azsci_topic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Elmi mətnlərin mövzular üzrə sinifləndirilməsi

Mən bu Jupyter dəftərində öyrədilmiş (pre-trained) transformer modelinin verilən
data toplusu əsasında mətn sinifləndirilməsi tapşırığı üçün sazlanma(fine-tune) prosesini göstərəcəm. Bunun üçün seçdiyim data toplusu Azərbaycan universitet və institutlarında müdafiə edilmiş dissertasiyaların başlıqları və onların aid olduğu elmi sahələrdir. Sinifləndirmə əməliyyatında başlıqlar verilən mətni, elmi sahələr isə mövzuları, yəni sinifləri ifadə edir.

Dəftərdə icra etdiyim əməliyyatları mərhələlər daha aydın bölünsün deyə 7 fərqli hissəyə ayırdım. Datanın yükləmək və prosesi, öyrədilmiş modelin təyini, sazlanması proseslərinin hamısı HuggingFace kitabxanalarının əsasında aparılmışdır.


## Kitabxanaların yüklənməsi

İlkin olaraq istifadə olunan kitabxanaları yükləmək lazımdır.

- Data üzərində əməliyyatlar üçün `datasets` kitabxanası
- Modelin təyini və sazlanma prosesi üçün `transformers[torch]` kitabxanası. Mən sazlanma əməliyyatlarını `PyTorch` əsaslı aparmaq istədiyimə görə `tranformers` modelinin `torch` əlavəsini istifadə etdim. Siz istəyinizə uyğun olaraq `Tensorflow` əsaslı versiyanı da istifadə edə bilərsiniz, ancaq ona uyğun olaraq göstərəcəyim kodun çox hissəsi dəyişməli olacaq.

In [ ]:
!pip install -U datasets
!pip install -U transformers[torch]

## İstifadə ediləcək metodların və verilənlərin təyini

In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
dataset = 'hajili/azsci_topics' # istifadə etdiyimiz HuggingFace data toplusu
model_name = 'FacebookAI/xlm-roberta-base' # öyrədilmiş modelin HuggingFace qovluğu
hf_output_directory = 'hajili/xlm-roberta-base-azsci-topics' # sazlanan modelin yükləyəcəyiniz qovluq

Sonda sazlanan modeli öz hesabınıza yükləmək üçün HuggingFace hesabınıza giriş edə bilərsiniz. `notebook_login()` metodu sizə HuggingFace tokeninizlə asan giriş etməyə imkan verir.

In [ ]:
notebook_login() # sonda sazlanan modeetmək üçün öz hesab

## Data əməliyyatları

Data toplusundakı dissertasiya başlıqları kifayət qədər təmiz olduğundan mətn üzərində minimal dəyişikliklər etdim. İlkin olaraq datanı yükləyib onu randomizasiya sabit qalsın deyə eyni `seed` ilə öyrənmə və test datalarına bölürük. Seçilən `42` adətən universal sabit kimi qəbul edilir, riyaziyyatla əlaqəli olmayan izah üçün: [42 və Avtostopçunun qalaktika bələdçisi](https://www.dictionary.com/e/slang/42/)

In [ ]:
data = load_dataset(dataset, split='train')
data = data.train_test_split(test_size=0.2, seed=42)
data = data.filter(lambda d: d['title']) # boş başlıqların təmizlənməsi.

train = data['train']
test = data['test']

In [ ]:
train

Dataset({
    features: ['title', 'topic', 'subtopic'],
    num_rows: 4604
})

In [ ]:
test

Dataset({
    features: ['title', 'topic', 'subtopic'],
    num_rows: 1152
})

Mövzu üzrə sinifləndirmə üçün nisbətən daha az sayda olan `topic` üzrə etdim. `subtopic` yəni alt mövzular iyerarxik sinifləndirmə üzrə oxucu üçün maraqlı tapşırıq ola bilər.

Sinifləri müəyyən etmək üçün data toplusundakı bütün mövzuları təyin etmək lazımdır. Aşağıdakı əməliyyat data toplusundakı bütün mövzuları əlifba ardıcılığı ilə sıralayır.

In [ ]:
all_topics = sorted(list(set(train['topic']) | set(test['topic'])))
all_topics

['Aqrar elmlər',
 'Astronomiya',
 'Biologiya elmləri',
 'Coğrafiya',
 'Filologiya elmləri',
 'Fizika',
 'Fəlsəfə',
 'Hüquq elmləri',
 'Kimya',
 'Memarlıq',
 'Mexanika',
 'Pedaqogika',
 'Psixologiya',
 'Riyaziyyat',
 'Siyasi elmlər',
 'Sosiologiya',
 'Sənətşünaslıq',
 'Tarix',
 'Texnika elmləri',
 'Tibb elmləri',
 'Yer elmləri',
 'İqtisad elmləri',
 'Əczaçılıq elmləri']

Əlavə olaraq sazlama zamanı sinifləri `[0, #Mövzu-1]` aralığına uyğun təyin etdiyimizə görə mövzuları öz indekslərinə uyğunlaşdırırıq.

In [ ]:
topic_to_label = {k: v for v, k in enumerate(all_topics)}
topic_to_label

{'Aqrar elmlər': 0,
 'Astronomiya': 1,
 'Biologiya elmləri': 2,
 'Coğrafiya': 3,
 'Filologiya elmləri': 4,
 'Fizika': 5,
 'Fəlsəfə': 6,
 'Hüquq elmləri': 7,
 'Kimya': 8,
 'Memarlıq': 9,
 'Mexanika': 10,
 'Pedaqogika': 11,
 'Psixologiya': 12,
 'Riyaziyyat': 13,
 'Siyasi elmlər': 14,
 'Sosiologiya': 15,
 'Sənətşünaslıq': 16,
 'Tarix': 17,
 'Texnika elmləri': 18,
 'Tibb elmləri': 19,
 'Yer elmləri': 20,
 'İqtisad elmləri': 21,
 'Əczaçılıq elmləri': 22}

Data toplusunda indekslərin əsasında `label` adlı yeni verilən təyin edirik. `label` sazlama zamanı modelin çıxış qatındakı nəticənin indeksini bildirəcək.

In [ ]:
train = train.map(lambda t: {'label': topic_to_label[t['topic']]})
test = test.map(lambda t: {'label': topic_to_label[t['topic']]})

In [ ]:
train[0]

{'title': 'Azərbaycanda yayılan toksigen göbələklərin ekobioloji xüsusiyyətləri',
 'topic': 'Biologiya elmləri',
 'subtopic': 'Mikrobiologiya',
 'label': 2}

In [ ]:
test[0]

{'title': 'Külək mühərriklərinin multiplikatorunun axtarışla konstruksiya edilməsi',
 'topic': 'Texnika elmləri',
 'subtopic': 'Maşinlar, avadanlıqlar və proseslər',
 'label': 18}

## Tokenlərə bölünmə

Tokenlərə bölünmə üçün istifadə etdiyimiz öyrədilmiş modelin öz token konfiqurasiyasını işlədirik.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_data(data):
    return tokenizer(data["title"], truncation=True)

In [ ]:
train = train.map(tokenize_data)
test = test.map(tokenize_data)

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

In [ ]:
test

Dataset({
    features: ['title', 'topic', 'subtopic', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1152
})

Adətən data toplusunda mətn uzunluğu və buna uyğun olaraq token sayı müxtəlif saylarda olur. Modelin sazlanması prosesi datanı hissələrə (`batch`) bölərək icra edilir. Hər hansı verilən bir hissədə token saylarını eyniləşdirmək üçün `padding` yəni təyin edilmiş eyni tokenləri əlavə etmə əməliyyatı edilir. Bunu təyin etmək üçün `transformer` kitabxanasında `DataCollatorWithPadding` modulundan istifadə etmək olar.

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Sazlama


İlkin olaraq modeli təyin edirik. Əsas olaraq götürdüyümüz modellər adətən çoxdilli data ilə öyrədilir. Mənim bildiyim sırf Azərbaycan dili üçün öyrədilmiş yaxşı işləyən transformer əsaslı model yoxdu. Ona görə mən eksperimentlərimdə `xlm-roberta-base`, `xlm-roberta-large` və `mdeberta-v3-base` modellərindən istifadə etdim.


`AutoModelForSequenceClassification` öyrədilmiş modelin üzərində sinifləndirmə modulu ilə birlikdə təyin edilməsinə və son qatında sinif sayına uyğun olaraq neçə neyronun olacağının göstərilməsinə imkan verir.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(all_topics))

Sazlama zamanı modelin perfomansının ölçülməsi üçün `compute_metrics` metodu təyin edirik. Bu metod ilə `precision`, `recall`, `f1` və `accuracy` dəyərlərini hesablayırıq. Bu dəyərlərin ilk üçünü siniflərin data toplusunda saylarına uyğun olaraq mütənasib hesablanması üçün `weighted` ortalama üsulundan istifadə edə bilərik.

In [ ]:
pre = load_metric("precision", trust_remote_code=True)
rec = load_metric("recall", trust_remote_code=True)
f1_ = load_metric("f1", trust_remote_code=True)
acc = load_metric("accuracy", trust_remote_code=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = pre.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = rec.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    res = {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}
    return res


Sazlama arqumentlərinin təyini adətən eksperimentlərə uyğun olaraq aparılır, siz aşağıda verilənləri dəyişib yoxlaya da bilərsiniz.
- Əməliyyatı icra etdiyiniz cihazın (GPU) yaddaşına uyğun olaraq `per_device_train_batch_size` və `per_device_eval_batch_size` dəyişib yoxlamaq olar.
- Mənim eksperimentlərimdə bu data toplusunda əksər öyrədilmiş modellərdə 5 epoxadan sonra həddindən artıq öyrənmə baş verir.(`overfitting` demək istəyirəm, gülməyin)
- Ən yaxşı modeli saxlamaq üçün `metric_for_best_model` və `greater_is_better` istifadə oluna bilər.

In [ ]:
training_args = TrainingArguments(
    output_dir=f'./xlm-roberta-large-azsci-topics',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## Qiymətləndirmə

Sazlanmış modeli qiymətləndirmək üçün `.predict()` metodu ilə test data toplusunda sinifləndirməni icra etmək və doğru siniflər əsaslında nəticələri yoxlamaq lazımdır.

In [ ]:
training_results = trainer.predict(test)

In [ ]:
logits, labels = training_results.predictions, training_results.label_ids
predictions = np.argmax(logits, axis=-1)

`sklearn` kitabxanasının `classification_report` metodu verilən təxminlərin və doğru siniflərin əsasında hər sinifə uyğun və ümumi nəticələri əldə etməyə imkan verir.

In [ ]:
report = classification_report(labels, predictions, output_dict=True)

In [ ]:
class_names, pres, recs, f1s, sups = [], [], [], [], []
for c_res in report:
  class_name = c_res
  try:
    class_name = all_topics[int(c_res)]
  except:
    pass
  if 'accuracy' != class_name:
    class_names.append(class_name)
    pres.append(report[c_res]['precision'])
    recs.append(report[c_res]['recall'])
    f1s.append(report[c_res]['f1-score'])
    sups.append(report[c_res]['support'])

In [ ]:
df = pd.DataFrame([class_names, pres, recs, f1s, sups])
df = df.transpose()
df.columns=['Topic', 'Precision', 'Recall', 'F1', 'Support']

In [ ]:
df.to_markdown(index=False)

| Topic              |   Precision |   Recall |       F1 |   Support |
|:-------------------|------------:|---------:|---------:|----------:|
| Aqrar elmlər       |    0.703704 | 0.703704 | 0.703704 |        27 |
| Astronomiya        |    0        | 0        | 0        |         2 |
| Biologiya elmləri  |    0.886598 | 0.819048 | 0.851485 |       105 |
| Coğrafiya          |    0.75     | 0.705882 | 0.727273 |        17 |
| Filologiya elmləri |    0.91954  | 0.914286 | 0.916905 |       175 |
| Fizika             |    0.710526 | 0.794118 | 0.75     |        34 |
| Fəlsəfə            |    0.7      | 0.5      | 0.583333 |        14 |
| Hüquq elmləri      |    1        | 1        | 1        |        29 |
| Kimya              |    0.75     | 0.934426 | 0.832117 |        61 |
| Memarlıq           |    1        | 0.4      | 0.571429 |         5 |
| Mexanika           |    0        | 0        | 0        |         4 |
| Pedaqogika         |    0.854545 | 1        | 0.921569 |        47 |
| Psix

## Modeldən istifadə

Sazlama bitdikdən sonra əldə etdiyiniz modeli HuggingFace hesabınıza `.push_to_hub()` metodu ilə yükləyə bilərsiniz.

In [ ]:
trainer.push_to_hub(hf_output_directory)

`transformers` kitabxanasının `pipeline` modulu sazlanmış modeli sinifləndirmə əməliyyatı üçün asanlıqla istifadə etməyə kömək edir.

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=hf_output_directory)

In [ ]:
pipe("XX əsrdə ABŞ-Çin münasibətlərinin təhlili")